In [1]:
import DifferentialEquations as DE
import ModelingToolkit as MTK
import Plots
import ModelingToolkit: t_nounits as t, D_nounits as D, @variables, @parameters, @named, @mtkbuild, @register_symbolic

In [2]:
# Q10. MTK ver
using CoolProp, ModelingToolkit, DifferentialEquations
# CoolProp 함수들을 MTK에 등록합니다. (추후 Tabular 값으로 변경)
 # ρ, T 등을 P, h에서 계산하는 함수
R410a_density(p, h) = PropsSI("Dmass", "P", p, "Hmass", h, "R410A")
R410a_temperature(p, h) = PropsSI("T", "P", p, "Hmass", h, "R410A")

@register_symbolic R410a_density(p, h)
@register_symbolic R410a_temperature(p, h)

Symbolics.derivative(::typeof(R410a_density), args::NTuple{2,Any}, ::Val{1}) = R410a_density(args[1]+1, args[2]) -  R410a_density(args[1], args[2])
Symbolics.derivative(::typeof(R410a_density), args::NTuple{2,Any}, ::Val{2}) = R410a_density(args[1], args[2]+1) -  R410a_density(args[1], args[2])

In [ ]:
function CV_N_problem(;N = 10)
    # 1. Params for Ref
    @parameters V_elem = 0.02*0.02 / 10 T_wall = 40 + 273.15 UA_elem = 8000 * 0.02 / 10
    @parameters c1 = 9 * 1e-6 c2 = 9 * 1e-6
    @parameters P_in = 3200e3 P_out = 3000e3 h_in = 450e3 ρ_in = R410a_density(P_in, h_in)
    
    # 1. Variables for Ref
    @variables P(t) = P_in h(t)[1:N] = h_in*ones(N) ρ(t)[1:N] T(t)[1:N]
    @variables mdot_in(t)[1:N] mdot_out(t)[1:N] Q_r(t)[1:N] h_out(t)[1:N]

    eva_ref = [
        [    
        # 1-1. Algebraic eqns for Ref
        ρ[i] ~ R410a_density(P, h[i]),
        T[i] ~ R410a_temperature(P, h[i]),
        mdot_in[i] ~ (i == 1 ? c1 * sqrt(max(0.0, ρ_in * (P_in - P))) : mdot_out[i-1]),
        mdot_out[i] ~ (i == N ? c2 * sqrt(max(0.0, ρ[i] * (P - P_out))) : mdot_in[i+1]),
        h_out[i] ~ h[i],
        Q_r[i] ~ UA_elem * (T_wall - T[i]),

        # 1-2. Differential eqns for Ref
        V_elem * D(ρ[i]) ~ mdot_in[i] - mdot_out[i],
        V_elem * h[i] * D(ρ[i]) + V_elem * ρ[i] * D(h[i]) - V_elem * D(P) ~ mdot_in[i] * (i == 1 ? h_in : h_out[i-1]) - mdot_out[i] * h_out[i] + Q_r[i]
        ]
        for i in 1:N
    ]

    eqns = vcat(eva_ref...)

    @mtkbuild sys = MTK.ODESystem(eqns, t)
    return eqns, sys
end

function solve_CV_N_problem(;N = 10,tspan = (0.0, 10.0))
    # Convert from a symbolic to a numerical problem to simulate
    sys = CV_N_problem(N=N)
    prob = DE.ODEProblem(sys, [], tspan)
    # Solve the ODE
    sol = DE.solve(prob, Rodas5P(autodiff=false))

    # sys에서 P와 h 변수를 가져옵니다.
    @unpack P, h = sys

    # 1. 각 h[i]에 대한 솔루션 벡터를 리스트로 만듭니다.
    all_h_sols = [sol[h[i]] for i in 1:N]

    # 2. P의 솔루션 벡터와 h의 솔루션 리스트를 vcat으로 합칩니다.
    # 결과: [Vector(P), Vector(h1), Vector(h2), ...] 형태의 Vector{Vector}
    return vcat([sol.t], [sol[P]], all_h_sols)
end

solve_CV_N_problem (generic function with 1 method)

In [4]:
function plot_the_res(sol, N=10)
    # 1. 데이터 추출 (솔루션 구조에 맞게 수정)
    sol_t = sol[1]
    P_sol = sol[2]
    # h_sol은 h1, h2, ... 의 결과 벡터들을 담은 리스트가 됩니다.
    h_sols = sol[3:(2+N)]

    # 물리량 정의
    V_elem = 0.02 * 0.02 / N  # 각 요소(node)의 부피
    UA_elem = 8000 * 0.02 / N # 각 요소의 총괄열전달계수
    T_wall = 40.0             # 벽면 온도 (섭씨)

    # 2. 결과를 저장할 배열 초기화 (시간 스텝 길이에 맞춤)
    num_timesteps = length(sol_t)
    mtot = zeros(num_timesteps)
    Q_tot = zeros(num_timesteps)
    T_avg = zeros(num_timesteps) # 평균 온도를 저장할 배열

    # 3. 시간을 기준으로 외부 반복 실행
    for j in 1:num_timesteps
        # 각 시간 스텝(j)에서의 총합을 계산하기 위한 임시 변수
        m_sum_at_j = 0.0
        Q_sum_at_j = 0.0
        T_sum_at_j = 0.0

        # 내부 반복: 해당 시간의 모든 노드(i)에 대해 계산
        for i in 1:N
            # 시간 j, 노드 i에서의 압력과 엔탈피
            p_ij = P_sol[j]
            h_ij = h_sols[i][j]

            # 해당 지점의 밀도와 온도 계산
            rho_ij = R410a_density(p_ij, h_ij)
            T_ij_celsius = R410a_temperature(p_ij, h_ij) - 273.15

            # 해당 노드의 질량과 열전달량 계산
            m_ij = rho_ij * V_elem
            Q_ij = UA_elem * (T_wall - T_ij_celsius)

            # 총합에 더하기
            m_sum_at_j += m_ij
            Q_sum_at_j += Q_ij
            T_sum_at_j += T_ij_celsius
        end

        # 해당 시간 스텝(j)의 최종 계산값 저장
        mtot[j] = m_sum_at_j
        Q_tot[j] = Q_sum_at_j
        T_avg[j] = T_sum_at_j / N # 평균 온도 계산
    end

    # 4. 결과 시각화
    p1 = Plots.plot(sol_t, P_sol ./ 1e3, label="Pressure (P)", xlabel="Time (s)", ylabel="kPa")
    p2 = Plots.plot(sol_t, mtot, label="Total Mass (m_tot)", xlabel="Time (s)", ylabel="kg")
    p3 = Plots.plot(sol_t, T_avg, label="Average Temperature (T_avg)", xlabel="Time (s)", ylabel="°C")
    p4 = Plots.plot(sol_t, Q_tot, label="Total Heat Transfer (Q_tot)", xlabel="Time (s)", ylabel="W")

    return Plots.plot(p1, p2, p3, p4, layout=(2, 2), legend=:best)
end

plot_the_res (generic function with 2 methods)

In [12]:
eqs, sys = CV_N_problem(N=10)
eqs

80-element Vector{Equation}:
 (ρ(t))[1] ~ Main.R410a_density(P(t), (h(t))[1])
 (T(t))[1] ~ Main.R410a_temperature(P(t), (h(t))[1])
 (mdot_in(t))[1] ~ c1*sqrt(max(0.0, (P_in - P(t))*ρ_in))
 (mdot_out(t))[1] ~ (mdot_in(t))[2]
 (h_out(t))[1] ~ (h(t))[1]
 (Q_r(t))[1] ~ (-(T(t))[1] + T_wall)*UA_elem
 V_elem*Differential(t)((ρ(t))[1]) ~ (mdot_in(t))[1] - (mdot_out(t))[1]
 -V_elem*Differential(t)(P(t)) + V_elem*(h(t))[1]*Differential(t)((ρ(t))[1]) + V_elem*Differential(t)((h(t))[1])*(ρ(t))[1] ~ (Q_r(t))[1] + h_in*(mdot_in(t))[1] - (h_out(t))[1]*(mdot_out(t))[1]
 (ρ(t))[2] ~ Main.R410a_density(P(t), (h(t))[2])
 (T(t))[2] ~ Main.R410a_temperature(P(t), (h(t))[2])
 ⋮
 -V_elem*Differential(t)(P(t)) + V_elem*(h(t))[9]*Differential(t)((ρ(t))[9]) + V_elem*Differential(t)((h(t))[9])*(ρ(t))[9] ~ (Q_r(t))[9] + (h_out(t))[8]*(mdot_in(t))[9] - (h_out(t))[9]*(mdot_out(t))[9]
 (ρ(t))[10] ~ Main.R410a_density(P(t), (h(t))[10])
 (T(t))[10] ~ Main.R410a_temperature(P(t), (h(t))[10])
 (mdot_in(t))[10] ~ (mdot_

In [ ]:
sol = solve_CV_N_problem(N=10, tspan = (0.0, 10.0))
plot_the_res(sol)